Desarrollo Algoritmo para Prueba 1

1) Selección de Países y Recolección de Datos
Esta celda procesa los datos del recurso solar y condiciones meteorológicas de Chile, Sudáfrica e Indonesia.

In [4]:
import polars as pl
from pathlib import Path

# Definir los pares de archivos por ubicación
ubicaciones = {
    'Sudáfrica': {
        'lat': -29.19,
        'lon': 21.30,
        'archivos': ['1155731_-29.19_21.30_2018.csv', '1155731_-29.19_21.30_2019.csv']
    },
    'China': {
        'lat': 44.73,
        'lon': 87.66,
        'archivos': ['3480150_44.73_87.66_2018.csv', '3480150_44.73_87.66_2019.csv']
    },
    'Chile': {
        'lat': -23.84,
        'lon': -69.89,
        'archivos': ['5815755_-23.84_-69.89_2018.csv', '5815755_-23.84_-69.89_2019.csv']
    }
}

# Procesar cada ubicación
for id_ubicacion, info in ubicaciones.items():
    print(f"\nProcesando ubicación {id_ubicacion} ({info['lat']}, {info['lon']})")
    
    # Lista para almacenar los DataFrames de cada año
    dfs = []
    
    # Procesar cada archivo de la ubicación
    for archivo in info['archivos']:
        # Leer el archivo CSV, saltando las primeras 2 filas que contienen metadatos
        df = pl.read_csv(archivo, skip_rows=2)
        # Filtrar las columnas que NO contengan 'Clearsky'
        columnas_filtradas = [col for col in df.columns if 'Clearsky' not in col]
        df = df.select(columnas_filtradas)
        # Agregar columnas de ubicación
        df = df.with_columns([
            pl.lit(id_ubicacion).alias('Location_ID'),
            pl.lit(info['lat']).alias('Latitude'),
            pl.lit(info['lon']).alias('Longitude')
        ])
        
        # Agregar el DataFrame a la lista
        dfs.append(df)
    
    # Combinar los DataFrames de los dos años
    df_final = pl.concat(dfs)
    
    # Ordenar por fecha y hora
    df_final = df_final.sort(['Year', 'Month', 'Day', 'Hour', 'Minute'])
    
    # Mostrar las primeras filas del DataFrame
    print(f"\nPrimeras filas del DataFrame para ubicación {id_ubicacion}:")
    print(df_final.head())
    
    # Mostrar información básica del DataFrame
    print(f"\nInformación del DataFrame para ubicación {id_ubicacion}:")
    print(df_final.describe())
    
    # Crear directorio para los resultados si no existe
    Path('resultados').mkdir(exist_ok=True)
    
    # Guardar el DataFrame en un nuevo archivo CSV
    nombre_archivo = f'resultados/datos_{id_ubicacion}.csv'
    df_final.write_csv(nombre_archivo)
    print(f"\nLos datos han sido guardados en '{nombre_archivo}'")

print("\nProcesamiento completado para todas las ubicaciones")
print("\nPara ver el resultado completo, seleccione view as scrollable element")

#pl.Config.set_fmt_str_lengths(10000)
#pl.Config.set_tbl_rows(1000)


Procesando ubicación Sudáfrica (-29.19, 21.3)

Primeras filas del DataFrame para ubicación Sudáfrica:
shape: (5, 19)
┌──────┬───────┬─────┬──────┬───┬───────────────────┬─────────────┬──────────┬───────────┐
│ Year ┆ Month ┆ Day ┆ Hour ┆ … ┆ Relative Humidity ┆ Location_ID ┆ Latitude ┆ Longitude │
│ ---  ┆ ---   ┆ --- ┆ ---  ┆   ┆ ---               ┆ ---         ┆ ---      ┆ ---       │
│ i64  ┆ i64   ┆ i64 ┆ i64  ┆   ┆ f64               ┆ str         ┆ f64      ┆ f64       │
╞══════╪═══════╪═════╪══════╪═══╪═══════════════════╪═════════════╪══════════╪═══════════╡
│ 2018 ┆ 1     ┆ 1   ┆ 0    ┆ … ┆ 64.11             ┆ Sudáfrica   ┆ -29.19   ┆ 21.3      │
│ 2018 ┆ 1     ┆ 1   ┆ 1    ┆ … ┆ 66.25             ┆ Sudáfrica   ┆ -29.19   ┆ 21.3      │
│ 2018 ┆ 1     ┆ 1   ┆ 2    ┆ … ┆ 69.09             ┆ Sudáfrica   ┆ -29.19   ┆ 21.3      │
│ 2018 ┆ 1     ┆ 1   ┆ 3    ┆ … ┆ 70.29             ┆ Sudáfrica   ┆ -29.19   ┆ 21.3      │
│ 2018 ┆ 1     ┆ 1   ┆ 4    ┆ … ┆ 66.65             ┆ Sudáfrica

Analisis de Recurso Solar

In [7]:
import polars as pl
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import os

# Configuración del estilo
plt.style.use('default')


# Cargar los datos usando Polars
def cargar_datos(archivo):
    return pl.read_csv(archivo)

# Cargar los datos de cada país
datos_chile = cargar_datos('resultados/datos_Chile.csv')
datos_china = cargar_datos('resultados/datos_China.csv')
datos_sudafrica = cargar_datos('resultados/datos_Sudáfrica.csv')

# Convertir las columnas de fecha y hora a datetime
def procesar_fecha_hora(df):
    return df.with_columns([
        pl.col('Year').cast(pl.Int64),
        pl.col('Month').cast(pl.Int64),
        pl.col('Day').cast(pl.Int64),
        pl.col('Hour').cast(pl.Int64),
        pl.col('Minute').cast(pl.Int64)
    ]).with_columns([
        pl.datetime(
            'Year', 'Month', 'Day', 'Hour', 'Minute'
        ).alias('datetime')
    ])

# Procesar las fechas para cada dataset
datos_chile = procesar_fecha_hora(datos_chile)
datos_china = procesar_fecha_hora(datos_china)
datos_sudafrica = procesar_fecha_hora(datos_sudafrica)

# Calcular promedios e irradiación diarios
def calcular_promedios_diarios(df):
    # Factor de conversión: (W/m² * minuto) a (kWh/m²)
    # 1 kWh/m² = 1000 W/m² * 1 hora
    # Como tenemos datos cada minuto, multiplicamos por (1/60) para convertir a horas
    # y por (1/1000) para convertir a kWh
    factor_conversion = 1 / (60 * 1000)
    
    return df.group_by_dynamic(
        'datetime',
        every='1d',
        period='1d'
    ).agg([
        pl.col('GHI').mean().alias('GHI_promedio'),
        pl.col('DHI').mean().alias('DHI_promedio'),
        pl.col('DNI').mean().alias('DNI_promedio'),
        (pl.col('GHI').sum() * factor_conversion).alias('GHI_irradiacion'),
        (pl.col('DHI').sum() * factor_conversion).alias('DHI_irradiacion'),
        (pl.col('DNI').sum() * factor_conversion).alias('DNI_irradiacion')
    ])

# Calcular promedios diarios para cada país
promedios_chile = calcular_promedios_diarios(datos_chile)
promedios_china = calcular_promedios_diarios(datos_china)
promedios_sudafrica = calcular_promedios_diarios(datos_sudafrica)

# Crear gráficas
def crear_graficas_irradiancia():
    plt.figure(figsize=(15, 10))

    # Graficar GHI
    plt.subplot(3, 1, 1)
    plt.plot(promedios_chile['datetime'], promedios_chile['GHI_promedio'], label='Chile', linewidth=2)
    plt.plot(promedios_china['datetime'], promedios_china['GHI_promedio'], label='China', linewidth=2)
    plt.plot(promedios_sudafrica['datetime'], promedios_sudafrica['GHI_promedio'], label='Sudáfrica', linewidth=2)
    plt.title('Comparación de Irradiancia Global Horizontal (GHI)')
    plt.ylabel('GHI (W/m²)')
    plt.legend()
    plt.grid(True)

    # Graficar DHI
    plt.subplot(3, 1, 2)
    plt.plot(promedios_chile['datetime'], promedios_chile['DHI_promedio'], label='Chile', linewidth=2)
    plt.plot(promedios_china['datetime'], promedios_china['DHI_promedio'], label='China', linewidth=2)
    plt.plot(promedios_sudafrica['datetime'], promedios_sudafrica['DHI_promedio'], label='Sudáfrica', linewidth=2)
    plt.title('Comparación de Irradiancia Difusa Horizontal (DHI)')
    plt.ylabel('DHI (W/m²)')
    plt.legend()
    plt.grid(True)

    # Graficar DNI
    plt.subplot(3, 1, 3)
    plt.plot(promedios_chile['datetime'], promedios_chile['DNI_promedio'], label='Chile', linewidth=2)
    plt.plot(promedios_china['datetime'], promedios_china['DNI_promedio'], label='China', linewidth=2)
    plt.plot(promedios_sudafrica['datetime'], promedios_sudafrica['DNI_promedio'], label='Sudáfrica', linewidth=2)
    plt.title('Comparación de Irradiancia Directa Normal (DNI)')
    plt.xlabel('Fecha')
    plt.ylabel('DNI (W/m²)')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('resultados/comparacion_irradiancias.png', dpi=300, bbox_inches='tight')
    plt.close()

def crear_graficas_irradiacion():
    plt.figure(figsize=(15, 10))

    # Graficar GHI
    plt.subplot(3, 1, 1)
    plt.plot(promedios_chile['datetime'], promedios_chile['GHI_irradiacion'], label='Chile', linewidth=2)
    plt.plot(promedios_china['datetime'], promedios_china['GHI_irradiacion'], label='China', linewidth=2)
    plt.plot(promedios_sudafrica['datetime'], promedios_sudafrica['GHI_irradiacion'], label='Sudáfrica', linewidth=2)
    plt.title('Comparación de Irradiación Global Horizontal (GHI)')
    plt.ylabel('GHI (kWh/m²/día)')
    plt.legend()
    plt.grid(True)

    # Graficar DHI
    plt.subplot(3, 1, 2)
    plt.plot(promedios_chile['datetime'], promedios_chile['DHI_irradiacion'], label='Chile', linewidth=2)
    plt.plot(promedios_china['datetime'], promedios_china['DHI_irradiacion'], label='China', linewidth=2)
    plt.plot(promedios_sudafrica['datetime'], promedios_sudafrica['DHI_irradiacion'], label='Sudáfrica', linewidth=2)
    plt.title('Comparación de Irradiación Difusa Horizontal (DHI)')
    plt.ylabel('DHI (kWh/m²/día)')
    plt.legend()
    plt.grid(True)

    # Graficar DNI
    plt.subplot(3, 1, 3)
    plt.plot(promedios_chile['datetime'], promedios_chile['DNI_irradiacion'], label='Chile', linewidth=2)
    plt.plot(promedios_china['datetime'], promedios_china['DNI_irradiacion'], label='China', linewidth=2)
    plt.plot(promedios_sudafrica['datetime'], promedios_sudafrica['DNI_irradiacion'], label='Sudáfrica', linewidth=2)
    plt.title('Comparación de Irradiación Directa Normal (DNI)')
    plt.xlabel('Fecha')
    plt.ylabel('DNI (kWh/m²/día)')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('resultados/comparacion_irradiaciones.png', dpi=300, bbox_inches='tight')
    plt.close()

# Crear las gráficas
crear_graficas_irradiancia()
crear_graficas_irradiacion()

# Calcular y mostrar estadísticas
def calcular_estadisticas(df, pais):
    stats = df.select([
        pl.col('GHI').mean().alias('GHI_mean'),
        pl.col('GHI').max().alias('GHI_max'),
        pl.col('DHI').mean().alias('DHI_mean'),
        pl.col('DHI').max().alias('DHI_max'),
        pl.col('DNI').mean().alias('DNI_mean'),
        pl.col('DNI').max().alias('DNI_max')
    ])
    print(f"\nEstadísticas para {pais}:")
    print(stats)
    
    # Calcular estadísticas de irradiación
    irradiacion_stats = calcular_promedios_diarios(df).select([
        pl.col('GHI_irradiacion').mean().alias('GHI_irradiacion_media'),
        pl.col('GHI_irradiacion').max().alias('GHI_irradiacion_max'),
        pl.col('DHI_irradiacion').mean().alias('DHI_irradiacion_media'),
        pl.col('DHI_irradiacion').max().alias('DHI_irradiacion_max'),
        pl.col('DNI_irradiacion').mean().alias('DNI_irradiacion_media'),
        pl.col('DNI_irradiacion').max().alias('DNI_irradiacion_max')
    ])
    print(f"\nEstadísticas de irradiación diaria para {pais} (kWh/m²/día):")
    print(irradiacion_stats)

calcular_estadisticas(datos_chile, "Chile")
calcular_estadisticas(datos_china, "China")
calcular_estadisticas(datos_sudafrica, "Sudáfrica")

# Crear gráfica de barras para máximos
maximos = pl.DataFrame({
    'País': ['Chile', 'China', 'Sudáfrica'],
    'GHI': [
        datos_chile['GHI'].max(),
        datos_china['GHI'].max(),
        datos_sudafrica['GHI'].max()
    ],
    'DHI': [
        datos_chile['DHI'].max(),
        datos_china['DHI'].max(),
        datos_sudafrica['DHI'].max()
    ],
    'DNI': [
        datos_chile['DNI'].max(),
        datos_china['DNI'].max(),
        datos_sudafrica['DNI'].max()
    ]
})

plt.figure(figsize=(12, 6))
x = range(len(maximos['País']))
width = 0.25

plt.bar([i - width for i in x], maximos['GHI'], width, label='GHI')
plt.bar(x, maximos['DHI'], width, label='DHI')
plt.bar([i + width for i in x], maximos['DNI'], width, label='DNI')

plt.xlabel('País')
plt.ylabel('Irradiancia Máxima (W/m²)')
plt.title('Comparación de Máximos de Irradiancia por País')
plt.xticks(x, maximos['País'])
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('resultados/comparacion_maximos.png', dpi=300, bbox_inches='tight')
plt.close() 


Estadísticas para Chile:
shape: (1, 6)
┌────────────┬─────────┬───────────┬─────────┬────────────┬─────────┐
│ GHI_mean   ┆ GHI_max ┆ DHI_mean  ┆ DHI_max ┆ DNI_mean   ┆ DNI_max │
│ ---        ┆ ---     ┆ ---       ┆ ---     ┆ ---        ┆ ---     │
│ f64        ┆ i64     ┆ f64       ┆ i64     ┆ f64        ┆ i64     │
╞════════════╪═════════╪═══════════╪═════════╪════════════╪═════════╡
│ 291.582591 ┆ 1188    ┆ 48.764212 ┆ 583     ┆ 379.980137 ┆ 1098    │
└────────────┴─────────┴───────────┴─────────┴────────────┴─────────┘

Estadísticas de irradiación diaria para Chile (kWh/m²/día):
shape: (1, 6)
┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
│ GHI_irradiacio ┆ GHI_irradiacio ┆ DHI_irradiacio ┆ DHI_irradiaci ┆ DNI_irradiaci ┆ DNI_irradiaci │
│ n_media        ┆ n_max          ┆ n_media        ┆ on_max        ┆ on_media      ┆ on_max        │
│ ---            ┆ ---            ┆ ---            ┆ ---           ┆ ---           ┆ ---    

---------------------------------------------------------------------------------------------

2) Simulación de Plantas PV y CSP


Tecnología PV

In [16]:
#Simulación de producción anual de energía. 500kW, 1MW, 5MW
import PySAM
